In [0]:
from pyspark.sql import functions as F
from functools import reduce

# Diretório base contendo todos os arquivos Parquet consolidados (2020–2025)
base_dir = "/Volumes/workspace/nyc_taxi/raw/full"

# Schema alvo
target_schema = {
    "vendorid": "long",                 # ID do fornecedor/táxi 
    "tpep_pickup_datetime": "timestamp",# Data/hora do embarque
    "tpep_dropoff_datetime": "timestamp",# Data/hora do desembarque
    "passenger_count": "integer",       # Quantidade de passageiros (inteiro)
    "trip_distance": "double",          # Distância da viagem (milhas)
    "ratecodeid": "long",               # Código da tarifa aplicada
    "store_and_fwd_flag": "string",     # Flag de armazenamento/transmissão do dado
    "pulocationid": "long",             # ID da localização de embarque
    "dolocationid": "long",             # ID da localização de desembarque
    "payment_type": "long",             # Tipo de pagamento
    "fare_amount": "double",            # Valor da corrida sem taxas adicionais
    "extra": "double",                  # Valor de extras
    "mta_tax": "double",                # Taxa da MTA
    "tip_amount": "double",             # Valor de gorjeta
    "tolls_amount": "double",           # Valor de pedágios
    "improvement_surcharge": "double",  # Taxa de melhoria
    "total_amount": "double",           # Valor total da corrida
    "congestion_surcharge": "double",   # Taxa de congestionamento
    "airport_fee": "double",            # Taxa de aeroporto
}

target_cols = list(target_schema.keys())

# 📂 Lista todos os arquivos Parquet encontrados no diretório base
files = [f.path for f in dbutils.fs.ls(base_dir) if f.path.endswith(".parquet")]
assert files, f"Nenhum parquet encontrado em {base_dir}"
print(f"🔎 Encontrados {len(files)} arquivos")

def normalize_one(path: str):
    """
    Função responsável por:
    - Ler o arquivo Parquet individualmente.
    - Padronizar nomes de colunas para minúsculas.
    - Garantir que todas as colunas do schema alvo existam (mesmo que com NULL).
    - Realizar cast de cada coluna para o tipo correto.
    - Criar a coluna 'anomes' no formato YYYYMM a partir da data de pickup.
    - Retornar apenas colunas do schema alvo + partição.
    """
    df = spark.read.parquet(path)

    # Normaliza nomes de colunas para lowercase
    for c in df.columns:
        df = df.withColumnRenamed(c, c.lower())

    # Adiciona colunas ausentes e aplica cast para o tipo definido no schema alvo
    for col, dtype in target_schema.items():
        if col not in df.columns:
            df = df.withColumn(col, F.lit(None).cast(dtype))
        else:
            df = df.withColumn(col, F.col(col).cast(dtype))

    # Cria coluna de partição com base na data de pickup
    df = df.withColumn("anomes", F.date_format(F.col("tpep_pickup_datetime"), "yyyyMM"))

    # Seleciona somente colunas relevantes e descarta registros sem data válida
    return df.select(*target_cols, "anomes").filter(F.col("anomes").isNotNull())

# 🔄 Normaliza todos os arquivos individualmente
dfs = [normalize_one(p) for p in files]

# 🔗 Une todos os DataFrames normalizados de forma segura
df_all = reduce(lambda a, b: a.unionByName(b, allowMissingColumns=True), dfs)

# 🚮 Remove partições "outliers" com baixa contagem (menos de 10k linhas)
part_counts = df_all.groupBy("anomes").count().filter(F.col("count") >= 10000)
valid_partitions = [row["anomes"] for row in part_counts.collect()]
df_all = df_all.filter(F.col("anomes").isin(valid_partitions))

# 📊 Valida partições resultantes
print("📆 Partições encontradas:")
display(df_all.select("anomes").distinct().orderBy("anomes"))

# 🚮 Dropa a tabela anterior existente, para caso de mudança de schema
spark.sql("DROP TABLE IF EXISTS workspace.nyc_taxi.yellowtaxi_trips_sor")

# 💾 Escreve o resultado em uma tabela Delta Lake particionada por 'anomes'
(df_all.repartition("anomes")
      .write
      .format("delta")
      .mode("overwrite")
      .partitionBy("anomes")
      .saveAsTable("workspace.nyc_taxi.yellowtaxi_trips_sor"))

print(f"✅ Base SOR criada com sucesso! Total de linhas: {df_all.count()}")
